In [ ]:
%pip install profanity-check


In [5]:
from profanity_check import predict, predict_prob

text = "This is some bad profanity."
print(predict([text]))  # 1 means profane, 0 means clean
print(predict_prob([text]))  # Probability of profanity

ImportError: cannot import name 'joblib' from 'sklearn.externals' (p:\Programs\DSML\DSMLEnv\Lib\site-packages\sklearn\externals\__init__.py)

In [ ]:
%pip install joblib


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="unitary/unbiased-toxic-roberta")
text = "This is some bad profanity."
result = classifier(text)
print(result)  # Gives toxicity scores

In [ ]:
text = "This is some fucked profanity."
result = classifier(text)
print(result)  # Gives toxicity scores

In [ ]:
text = "fuck"
result = classifier(text)
print(result)  # Gives toxicity scores

In [ ]:
text = "Its all because of the cocks"  # some got dialogue
result = classifier(text)
print(result)  # Gives toxicity scores

In [ ]:
# lets try other ones
%pip install profanity-filter


In [ ]:
from profanity_filter import ProfanityFilter

pf = ProfanityFilter()
text = "This is some bad profanity."
print(pf.is_profane(text))  # Returns True if profane
print(pf.censor(text))  # Replaces profanity with ***

In [ ]:
%pip install better-profanity


In [ ]:
from better_profanity import profanity

text = "This is some bad profanity."
print(profanity.contains_profanity(text))  # True if profane
print(profanity.censor(text))  # Censors profane words

In [ ]:
text = "This is some fucked profanity."
print(profanity.contains_profanity(text))  # True if profane
print(profanity.censor(text))  # Censors profane words

In [ ]:
text = "This is some bullshit profanity."
print(profanity.contains_profanity(text))  # True if profane
print(profanity.censor(text))  # Censors profane words

In [ ]:
text = "Its all because of the cocks."
print(profanity.contains_profanity(text))  # True if profane
print(profanity.censor(text))  # Censors profane words

In [ ]:
%pip install nltk


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download("punkt_tab")

bad_words = {"badword1", "badword2"}  # Add your own list
text = "This is some badword1 in a sentence."

words = word_tokenize(text.lower())
contains_profanity = any(word in bad_words for word in words)

print(contains_profanity)  # True if profane words found""

In [ ]:
text = "This is some fucked word in a sentence."

words = word_tokenize(text.lower())
contains_profanity = any(word in bad_words for word in words)
print(contains_profanity)  # True if profane words found""

# so we got 2 working methods here

In [ ]:
def normal(text):
    print(profanity.contains_profanity(text))  # True if profane
    print(profanity.censor(text))  # Censors profane words


def ai(text):
    result = classifier(text)
    print(result)  # Gives toxicity scores

In [ ]:
def compare(text):
    print("normal")
    normal(text)
    print("ai")
    ai(text)

In [ ]:
compare("hello")

In [ ]:
compare("This is some bullshit")

In [ ]:
compare("This is some place the bulls have been using to shit")

In [ ]:
compare("its all because of the cocks")

In [ ]:
compare(" men doing all the fighting")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


def detect_profanity(text):
    """Detects whether the given text contains profanity."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return scores[0][1].item()  # Probability of being profane


# Example Usage
texts = [
    "You piece of shit!",  # English
]

for text in texts:
    score = detect_profanity(text)
    print(f"Text: {text} | Profanity Score: {score:.4f}")

In [ ]:
def compare(text):
    print("normal")
    normal(text)
    print("ai")
    print(detect_profanity(text))

In [ ]:
compare("fuck")

In [ ]:
%pip install safetext

In [8]:
from safetext import SafeText

st = SafeText(language="en")

In [9]:
results = st.check_profanity(
    text="Some text with a fucking word. Men are just cocks and dicks"
)
results

[{'word': 'cocks', 'index': 10, 'start': 44, 'end': 49},
 {'word': 'dicks', 'index': 12, 'start': 54, 'end': 59},
 {'word': 'fucking', 'index': 5, 'start': 17, 'end': 24}]

In [10]:
text = st.censor_profanity(
    text="Some text with a fucking word. Men are just cocks and dicks"
)
text

'Some text with a ******* word. Men are just ***** and *****'

In [12]:
# change profane sentence to sound natural

In [16]:
from transformers import pipeline

# Load OpenAI GPT model for paraphrasing
paraphraser = pipeline("text2text-generation", model="t5-small")


def smart_censor(text):
    """Replaces profane text with a grammatically correct alternative."""
    prompt = f"Paraphrase this without profanity: {text}"
    result = paraphraser(prompt, max_length=50, truncation=True)
    return result[0]["generated_text"]

Device set to use cuda:0


In [17]:
# Example
text = "This is fucking ridiculous!"
print(smart_censor(text))  # Example Output: "This is absolutely ridiculous!"

Paraphrase this without profanity: This is fucking ridiculous!


In [32]:
import ollama

#     prompt = f"Dont give any other response. Only give one response. Remove all profanity from this text while keeping the meaning intact.: '{text}'" works well

# also works fine Dont give any other response. Only give one response. Remove all profanity from the given text while keeping the meaning intact. If its just one word, replace with a more friendly word. Do not give advice. If it contains some sexual implication, try to translate to more kid friendly words. Here is the given text


def clean_text(text):
    prompt = f"""Return only the sanitized text. Remove all profanity while preserving the original meaning. If a single word is profane, replace it with a more appropriate alternative. For text with sexual implications, rewrite it in a kid-friendly manner. Do not provide explanations or advice. Only output the modified text. Give only 1 sentence
    
    Here are some examples of how to sanitize text:  

    Input: "Fuck you!"  
    Output: "Screw you!"  

    Input: "This is so fucked!"  
    Output: "This is a total mess!"  

    Input: "Men think with their dicks?"  
    Output: "Men often prioritize materialistic things?"  

    Input: "She was sniffing my ass!"  
    Output: "She was very close to me last night!"  

    Input: "I wanna fuck you so badly"  
    Output: "I want us to be close." 
    
    Input: "Have you never boned a woman's ass before jon snow?"  
    Output: "Have you never been close to a woman before, Jon Snow?" 
    

    Now, sanitize the following text while preserving its meaning: 
    
    : '{text}'"""
    response = ollama.chat(
        model="mistral", messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"]

In [25]:
clean_text("men think with their dicks")

' People often prioritize materialistic things.'

In [26]:
clean_text("You are a fucking idiotic whore")

'You are incredibly unwise.'

In [27]:
clean_text("i wanna fuck you")

' "I want to spend time with you."'

In [15]:
clean_text("She was sniffing my ass last night")

' She was very close to me last night.'

In [13]:
clean_text("fuck")

" : 'screw'"

In [30]:
clean_text("Dont your stones start to hurt if you dont use it? ")

"\nDoesn't using something regularly make it more comfortable?"

In [37]:
clean_text("Have you never boned a woman's ass before jon snow? ")

" 'Have you never had a romantic relationship with a woman before, Jon Snow?'"

In [40]:
clean_text("Fuck you!")

' "Screw you!"'